In [1]:
# PyAlex and Data Creation imports
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import pyalex, os, csv
pyalex.config.email = "b.cliff@gwmail.gwu.edu"
from itertools import chain

In [2]:
# Data Validation imports
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, List

#### Configuring Output Files

In [3]:
data_folder_loc = './data'

In [4]:
WORKS_FILE = f'{data_folder_loc}/works.csv'
AUTHORS_FILE = f'{data_folder_loc}/authors.csv'
TOPICS_FILE = f'{data_folder_loc}/topics.csv'
PUBLISHERS_FILE = f'{data_folder_loc}/publishers.csv'
FUNDERS_FILE = f'{data_folder_loc}/funders.csv'
WORK_AUTH_FILE = f'{data_folder_loc}/work_auth_edges.csv'
WORK_TOPIC_FILE = f'{data_folder_loc}/work_topic_edges.csv'
INSTITUTIONS_FILE = f'{data_folder_loc}/institutions.csv'
SOURCES_FILE = f'{data_folder_loc}/sources.csv'

In [5]:
# Removing any old/vestige files
for path in [WORKS_FILE, AUTHORS_FILE, TOPICS_FILE, WORK_AUTH_FILE,
             WORK_TOPIC_FILE, INSTITUTIONS_FILE, SOURCES_FILE]:
    if os.path.exists(path):
        os.remove(path)

In [6]:
works_file = open(WORKS_FILE, 'w', newline='',encoding='utf-8')
authors_file = open(AUTHORS_FILE, 'w', newline='', encoding='utf-8')
topics_file = open(TOPICS_FILE, 'w', newline='', encoding='utf-8')
institutions_file = open(INSTITUTIONS_FILE, 'w', newline='', encoding='utf-8')
publishers_file = open(PUBLISHERS_FILE, 'w', newline='', encoding='utf-8')
funders_file = open(FUNDERS_FILE, 'w', newline='', encoding='utf-8')
wa_edge_file = open(WORK_AUTH_FILE, 'w', newline='', encoding='utf-8')

In [7]:
wt_edge_file = open(WORK_TOPIC_FILE, 'w', newline='', encoding='utf-8')

In [8]:
# Creating the headers
works_writer = csv.writer(works_file)
authors_writer = csv.writer(authors_file)
topics_writer = csv.writer(topics_file)
institutions_writer = csv.writer(institutions_file)
publishers_writer = csv.writer(publishers_file)
funders_writer = csv.writer(funders_file)
wa_edges_writer = csv.writer(wa_edge_file)

In [9]:
wt_edges_writer = csv.writer(wt_edge_file)

In [10]:
# Writing the headers to the csv files
works_writer.writerow(['id', 'doi', 'title', 'publication_date', 'type', 'related_works', 'corresponding_author_ids'])
authors_writer.writerow(['id', 'name', 'works_count', 'cited_by_count', 'affiliations', 'topics', 'counts_by_year'])
institutions_writer.writerow(['id', 'display_name', 'ror', 'country_code', 'type', 'works_count', 'cited_by_count', 'homepage_url', 'is_super_system', 'created_date', 'updated_date'])
publishers_writer.writerow(['id', 'display_name', 'country_codes', 'hierarchy_level', 'parent_publisher', 'works_count', 'cited_by_count', 'created_date', 'updated_date'])
funders_writer.writerow(['id', 'display_name', 'description', 'country_code', 'grants_count', 'works_count', 'cited_by_count', 'homepage_url', 'created_date', 'updated_date'])
wa_edges_writer.writerow(['work_id', 'author_id', 'author_position', 'is_corresponding'])

52

In [11]:
wt_edges_writer.writerow(['work_id', 'topic_id', 'topic_name', 'score'])

35

In [12]:
topics_writer.writerow(['id', 'topic_name', 'description', 'keywords', 
                        'subfield', 'field', 'domain', 'works_count',
                        'cited_by_count' ,'updated_date' ,'created_date'])

111

#### Creating Pydantic Models for Validation

In [13]:
class WorksModel(BaseModel):
    id: str
    doi: Optional[str] = None
    title: str
    publication_date: str
    type: Optional[str] = None
    related_works: List[str] = None
    corresponding_author_ids: List[str] = None

In [14]:
class AuthorsModel(BaseModel):
    id: str
    display_name: str
    works_count: int
    cited_by_count: int
    affiliations: List[dict] = None
    topics: List[dict] = None
    counts_by_year: List[dict]

In [15]:
class TopicsModel(BaseModel):
    id: str
    display_name: str
    description: str
    keywords: List[str] = None
    subfield_name: Optional[str] = None
    field_name: Optional[str] = None
    domain_name: Optional[str] = None
    works_count: int
    cited_by_count: int
    updated_date: str
    created_date: str

In [16]:
class InstitutionsModel(BaseModel):
    id: str
    display_name: str
    ror: Optional[str] = None
    country_code: Optional[str] = None
    type: Optional[str] = None
    works_count: int
    cited_by_count: int
    homepage_url: Optional[str] = None
    is_super_system: bool
    created_date: str
    updated_date: str

In [17]:
class PublishersModel(BaseModel):
    id: str
    display_name: str
    country_codes: List[str] = None
    hierarchy_level: int
    parent_publisher: Optional[str] = None
    works_count: int
    cited_by_count: int
    created_date: str
    updated_date: str

In [18]:
class FundersModel(BaseModel):
    id: str
    display_name: str
    description: Optional[str] = None
    country_code: Optional[str] = None
    grants_count: int
    works_count: int
    cited_by_count: int
    homepage_url: Optional[str] = None
    created_date: str
    updated_date: str

### Data retrieval from OpenAlex

In [19]:
oa_replace = 'https://openalex.org/'

In [20]:
works_pager = Works().select(['id', 'doi', 'title',
'publication_date', 'type', 'related_works', 'corresponding_author_ids']).paginate(per_page=200)

In [21]:
authors_pager = Authors().select(['id', 'orcid', 'display_name',
                                  'works_count', 'cited_by_count',
                                  'affiliations', 'topics', 'counts_by_year']).paginate(per_page=200)

In [22]:
topics_pager = Topics().select(['id', 'display_name', 'description', 'keywords',
                                'subfield', 'field', 'domain', 'works_count',
                                'cited_by_count', 'updated_date', 'created_date']).paginate(per_page=200)
# not including siblings for now - complicated

In [23]:
institutions_pager = Institutions().select(['id', 'display_name', 'ror', 'country_code', 'type',
                                           'works_count', 'cited_by_count', 'homepage_url',
                                           'is_super_system', 'created_date', 'updated_date']).paginate(per_page=200)

In [24]:
publishers_pager = Publishers().select(['id', 'display_name', 'country_codes', 'hierarchy_level',
                                       'parent_publisher', 'works_count', 'cited_by_count',
                                       'created_date', 'updated_date']).paginate(per_page=200)

In [25]:
funders_pager = Funders().select(['id', 'display_name', 'description', 'country_code',
                                 'grants_count', 'works_count', 'cited_by_count',
                                 'homepage_url', 'created_date', 'updated_date']).paginate(per_page=200)

#### Data Cleaning

In [26]:
for page in works_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            raw['related_works'] = [rel.replace(oa_replace, '') for rel in raw['related_works']]
            raw['corresponding_author_ids'] = [rel.replace(oa_replace, '') for rel in raw['corresponding_author_ids']]
            w = WorksModel(**raw)
            works_writer.writerow(
                [w.id,
                w.doi,
                w.title,
                w.publication_date,
                w.type,
                w.related_works,
                w.corresponding_author_ids]
            )
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
works_file.close()


In [27]:
for page in authors_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            a = AuthorsModel(**raw)
            authors_writer.writerow([
                a.id,
                a.display_name,
                a.works_count,
                a.cited_by_count,
                a.affiliations,
                a.topics,
                a.counts_by_year
            ])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid record:", e)
            continue
authors_file.close()

In [28]:
for page in topics_pager:
    for raw in page:
        raw['id'] = raw['id'].replace(oa_replace, '')
        raw['subfield'] = raw['subfield'].get('display_name', '')
        raw['field'] = raw['field'].get('display_name', '')
        raw['domain'] = raw['domain'].get('display_name', '')
        t = TopicsModel(**raw)
        topics_writer.writerow([
            t.id,
            t.display_name,
            t.description,
            t.subfield_name,
            t.field_name,
            t.domain_name,
            t.works_count,
            t.cited_by_count,
            t.updated_date,
            t.created_date,
        ])
topics_file.close()

In [29]:
for page in institutions_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            # Extract ROR ID from the full URL if present
            if raw.get('ror'):
                raw['ror'] = raw['ror'].replace('https://ror.org/', '')
            i = InstitutionsModel(**raw)
            institutions_writer.writerow([...])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid institution record:", e)
            continue
institutions_file.close()


In [30]:
for page in publishers_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            # Clean parent publisher ID if present
            if raw.get('parent_publisher'):
                raw['parent_publisher'] = raw['parent_publisher'].replace(oa_replace, '')
            p = PublishersModel(**raw)
            publishers_writer.writerow([...])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid publisher record:", e)
            continue
publishers_file.close()


Skipping invalid publisher record: 1 validation error for PublishersModel
updated_date
  Field required [type=missing, input_value={'id': 'P4310320990', 'di...: '2023-01-01T00:00:00'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
Skipping invalid publisher record: 1 validation error for PublishersModel
updated_date
  Field required [type=missing, input_value={'id': 'P4310320595', 'di...: '2023-01-01T00:00:00'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


AttributeError: 'dict' object has no attribute 'replace'

In [ ]:
for page in funders_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            f = FundersModel(**raw)
            funders_writer.writerow([...])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid funder record:", e)
            continue
funders_file.close()


In [ ]:
for page in sources_pager:
    for raw in page:
        try:
            raw['id'] = raw['id'].replace(oa_replace, '')
            # Clean host organization ID if present
            if raw.get('host_organization'):
                raw['host_organization'] = raw['host_organization'].replace(oa_replace, '')
            s = SourcesModel(**raw)
            sources_writer.writerow([...])
        except (ValidationError, KeyError) as e:
            print("Skipping invalid source record:", e)
            continue
sources_file.close()


#### Including Filter and Select

#### Relationship Creation attempt

In [ ]:
class WorkAuthoredByEdge(BaseModel):
    work_id: str
    author_id: str
    author_position: str
    is_corresponding: bool

In [ ]:
class WorkHasTopic(BaseModel):
    work_id: str
    topic_id: str
    score: Optional[float] = None

class WorkCitesWork(BaseModel):
    citing_work_id: str
    cited_work_id: str

class AuthorHasTopic(BaseModel):
    author_id: str
    topic_id: str
    score: Optional[float] = None

In [ ]:
works_pager = Works().select(['id', 'authorships']).paginate(per_page=200)

In [ ]:
works_pager_topics = Works().select(['id', 'topics']).paginate(per_page=200)

In [ ]:
works_pager_citations = Works().select(['id', 'referenced_works']).paginate(per_page=200)

In [ ]:
for page in works_pager:
    for raw in page:
        work_id = raw.get('id')
        if not work_id:
            continue
        
        work_id = work_id.replace(oa_replace, '')

        for auth in raw.get('authorships') or []:
            author_block = auth.get("author") or {}
            author_id = author_block.get("id")

            if not author_id:
                continue

            try:
                edge = WorkAuthoredByEdge(
                    work_id=work_id,
                    author_id=author_id.replace(oa_replace, ""),
                    author_position=auth.get("author_position"),
                    is_corresponding=auth.get("is_corresponding")
                )

                wa_edges_writer.writerow([
                    edge.work_id,
                    edge.author_id,
                    edge.author_position,
                    edge.is_corresponding
                ])

            except ValidationError:
                continue


wa_edge_file.close()

In [ ]:
for page in works_pager_topics:
    for raw in page:
        work_id = raw.get('id')
        if not work_id:
            continue

        work_id = work_id.replace(oa_replace, '')

        for topic in raw['topics'] or []:
            topic_id = topic['id'].replace(oa_replace, '')
            topic_name = topic.get('display_name')
            topic_score = topic.get('score')

            wt_edges_writer.writerow([
                work_id,
                topic_id,
                topic_name,
                topic_score
            ])
wt_edge_file.close()            